
# Performance: American Monte Carlo and Multi-threading

To process realistic portfolios, ORE provides a few features that help keeping run times under control
1. **American Monte Carlo (AMC)** to accelerate exposure simulation for complex, but also vanilla products
2. **Multi-threading** to parallelise the NPV cube generation, utilising the CPU's cores

This notebook demonstrates both techniches and how they are configured in ORE.

Prerequisites:
- Python 3
- Jupyter
- ORE Python module installed: run "pip install open-source-risk-engine" to get the latest version

## Bermudan Swaption Performance: Classic vs AMC

Case:
- **Single Bermudan Swaption**, underlying Swap maturity 20y, 15 annual exercise dates from year 5 onwards. 
- Coarse simulation grid with 88 quarterly time steps

### Classic Simulation

- Generate the market paths by Monte Carlo simulation in the ORE Cross Asset Model
- The trade is priced under scenarios with a numerical engine (LGM grid engine in this case) which computes each Bermudan price in the cube with the usual backward induction on a numerical grid
- The pricing model is *not* recalibrated each time, just upfront
- **100 paths**, for moderate run time with the classic simulation in this demo

In [ ]:
from ORE import *
import sys, time, math
sys.path.append('..')
import utilities

params = Parameters()
params.fromFile("Input/ore_classic.xml")

ore = OREApp(params)

ore.run()

utilities.checkErrorsAndRunTime(ore)
utilities.writeList(ore.getReportNames())
keyNumber = 0 # key 0 corresponds to EONIA, compare to file header
numberOfPaths = 20
# Pick the t0 index fixing and pass it to the plot function below as starting point
analytic = ore.getAnalytic("XVA")
market = analytic.getMarket()
eoniaIndex = market.iborIndex("EUR-EONIA")
fixing = eoniaIndex.fixing(ore.getInputs().asof())

# Wait for the gzip file to be ready
utilities.wait_for_gzip_ready("Output/scenariodata.csv.gz")

utilities.plotScenarioDataPaths("Output/scenariodata.csv.gz", keyNumber, numberOfPaths, fixing)

In [ ]:
cubeReport = ore.getReport("netcube")
numberOfPaths = 20
utilities.plotNpvPaths(cubeReport, numberOfPaths)

In [ ]:
report = ore.getReport("exposure_trade_BermSwp")

time = report.dataAsReal(2)
epe = report.dataAsReal(3);
ene = report.dataAsReal(4);
    
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(10, 5))
gs = GridSpec(nrows=1, ncols=1)
ax0 = fig.add_subplot(gs[0, 0])

ax0.plot(time, epe, label='EPE')
ax0.plot(time, ene, label='ENE')
ax0.set(xlabel='Time/Years')
ax0.set(ylabel='Exposure')
ax0.set_title('Bermudan Exposure with 100 paths')
ax0.legend()

plt.show()

### American Monte Carlo Simulation

Leased Squares or American Monte Carlo algorithm in a nutshell:
- Generate Monte Carlo scenarios of the market evotion as in the classic simulation
- For the backward induction algorithm we then need continuation values on each path (conditional expectations of future values) and at each exercise time
- In the absence of a lattice, the AMC algortihm resorts to the information across paths and computes these conditional expectations via regression
- in ORE we use model states on the paths as regressors and perform linear regressions to determine the polynomial form of the regression fucntions  
- The valuation effort is determined by valuations of the underlying (vanilla) trade, plus the effort of estimating the regression functions and using them to compute conditional expectations
- We do two separate valuation sweeps, a first one for taining and determining the regression functions for each exercise date, a second sweep for the backward induction using the regression functions from sweep 1
- **10000** paths for training and backward induction (vs **100** paths in the classic simulation above)

In [ ]:
params_amc = Parameters()
params_amc.fromFile("Input/ore_amc.xml")
ore_amc = OREApp(params_amc, False)
      
ore_amc.run()

utilities.checkErrorsAndRunTime(ore_amc)

In [ ]:
utilities.writeList(ore_amc.getReportNames())

In [ ]:
report_amc = ore_amc.getReport("exposure_trade_BermSwp")

time = report_amc.dataAsReal(2)
epe = report_amc.dataAsReal(3);
ene = report_amc.dataAsReal(4);
    
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(10, 5))
gs = GridSpec(nrows=1, ncols=1)
ax0 = fig.add_subplot(gs[0, 0])

ax0.plot(time, epe, label='EPE')
ax0.plot(time, ene, label='ENE')
ax0.set(xlabel='Time/Years')
ax0.set(ylabel='Exposure')
ax0.set_title('Bermudan Exposure with 10,000 paths')
ax0.legend()

plt.show()

## Swap Portfolio Performance: Single vs. multi-threaded

Case
- Portfolio of **50** identical Swaps, 20 year maturity
- **Classic** exposure simulation
- **200 paths** (for moderate single-threaded run time)
- Simulation grid with 80 quarterly steps

### Single Thread

In [ ]:
params = Parameters()
params.fromFile("Input/ore_swaps.xml")

ore_swaps = OREApp(params, False)

ore_swaps.run()

utilities.checkErrorsAndRunTime(ore_swaps)

In [ ]:
portfolio = ore_swaps.getInputs().portfolio()
print("trades:", portfolio.size())
inputs = ore_swaps.getInputs()
print ("threads:", inputs.nThreads())

### Multiple Threads

ORE is based on QuantLib which is not thread-safe. Nevertheless, it is possible in ORE to parallelise the processing and utilise the available CPU cores (if you configure the ORE build with QL_ENABLE_SESSIONS=ON). 

All scenario-based risk analytics in ORE such as 
- Sensitivity analysis
- Historical simulation
- Monte Carlo simulation 

use a central ValuationEngine object which loops over scenarios, dates and trades and then applies pricing and cashflow analytics. The multi-threaded version of the ValuationEngine spawns threads, provides each thread with a **part of the portfolio** and its **own market objects**. The markets are built from scratch for each thread which causes some overhead. We plan to clone market objects instead to reduce the overhead. 

The threads then operate on completely separate objects, much like separate processes, because of QuantLib's inherent "thread-unsafety". Because of the overhead it is essential that each thread has a sufficiently sized or complex portfolio and sufficient number of scenarios to process, in order to see a multi-threading benefit. 

We demonstrate the benefit of multi-threading by running again with 4 threads:

In [ ]:
inputs.setThreads(4)
print ("threads:", inputs.nThreads())

In [ ]:
ore_swaps.run()
utilities.checkErrorsAndRunTime(ore_swaps)

## Bermudan Swaption Portfolio Performance: Single vs. multi-threaded

Case
- Portfolio of **20** identical Bermudan Swaptions
- **AMC** exposure simulation
- **10,000 paths**
- Simulation grid with 80 quarterly steps

In [ ]:
params = Parameters()
params.fromFile("Input/ore_bermudans.xml")

ore_bermudans = OREApp(params, False)

ore_bermudans.run()

utilities.checkErrorsAndRunTime(ore_bermudans)

In [ ]:
portfolio = ore_bermudans.getInputs().portfolio()
print("trades:", portfolio.size())
inputs = ore_bermudans.getInputs()
print ("threads:", inputs.nThreads())

In [ ]:
inputs.setThreads(4)
print ("threads:", inputs.nThreads())

In [ ]:
ore_bermudans.run()
utilities.checkErrorsAndRunTime(ore_bermudans)